In [35]:
re.sub('\n', '.', 'блиииин\nя дурак')

'блиииин.я дурак'

In [42]:
s = ['erg', 'QWEq']

In [56]:
from data4ml import Data4ML, Data4Chatbot
from bs4 import BeautifulSoup
import json
import os
import re
from perfect_regex import (
    perfect_url_regex,
    perfect_emoji_regex,
    perfect_email_regex,
    perfect_phone_regex,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
chatbot = Data4Chatbot(path_to_config='./../data_params.json')
chatbot.home_folder = "./data4test"
result = chatbot.make_data()

100%|██████████| 2/2 [00:00<00:00, 72.22it/s]


In [58]:
result[0]

['привет арин ты уже считала физику ? лабу',
 'привет нет еще',
 'хмм ну ладно я хотел графики сверить \n воин привет скинь пожалуйста контрольную по органу \n арин \n о спасибо',
 'сереж я вариант по органике но я тоже его сделала и переделывать не буду',
 'блиииин \n я дурак',
 'да ладно может ф .и не заметит что у нас одинаковые варианты',
 'он разрешил',
 'сереж ты спрашивал у екатерины валерьевны про домашку ?',
 'ну да \n она мне подсказывала',
 'ты можешь кинуть формулы если вы их вывели',
 'да но только как дома буду \n сейчас неудобно \n в общем мы кое что вывели',
 'ну это понятно',
 'она сказала ещё проверить \n ты точку эквивалентности уже считала ? \n потому что у тебя тоже возможно ошибка',
 'ну да я попыталась \n там корень из кс на \n ну как я понимаю',
 'а у тебя какая соль ? \n ну осадок',
 'хлорид кальция',
 'ну у меня наоборот \n в общем \n я как буду дома \n напишу \n арин \n привет \n извини что не написал \n я забыл совсем \n а ты не на понмила \n арин \n добрый 

In [13]:
chatbot = Data4Chatbot(path_to_config='../../../messages')

FileNotFoundError: [Errno 2] No such file or directory: '../../../messages'

In [120]:
list_of_files = chatbot.get_list_of_files_in_folder('data4test/153164714')
print(list_of_files)

['messages0.html']


In [121]:
all_messages = chatbot.parse_html('data4test/153164714', list_of_files)
all_messages

['Юлия Николаева, 23 мар 2016 в 20:48:28\nпришли фотки, пожалуйста',
 'Юлия Николаева, 23 мар 2016 в 20:48:55\nwrong@yandex.ru',
 'Вы, 23 мар 2016 в 21:51:32\nПривет, сейчас',
 'Вы, 23 мар 2016 в 21:53:35\nВсё👌',
 'Юлия Николаева, 23 мар 2016 в 22:50:04\nСпасибо большое',
 'Вы, 23 мар 2016 в 22:53:36\n👌',
 'Юлия Николаева, 20 апр 2016 в 10:39:48\nСережка, перешли мне письмо Смоленцевой.\n1 прикреплённое сообщение',
 'Юлия Николаева, 20 апр 2016 в 10:40:12\nЗаранее спасибо',
 'Вы, 20 апр 2016 в 10:47:08\nКакое именно?',
 'Юлия Николаева, 20 апр 2016 в 10:47:36\nПоследнее. Про кино',
 'Юлия Николаева, 20 апр 2016 в 10:48:44\nСпасибо',
 'Вы, 20 апр 2016 в 10:48:44\nСкинул',
 'Вы, 20 апр 2016 в 22:16:23\nПодойдите завтра к смоленцевой',
 'Юлия Николаева, 20 апр 2016 в 22:20:43\nзачем?',
 'Вы, 21 апр 2016 в 5:26:20\nОна попросила',
 'Юлия Николаева, 21 апр 2016 в 8:37:28\nХорошо',
 'Вы, 22 апр 2016 в 18:36:12\nЮль',
 'Вы, 22 апр 2016 в 18:36:12\nне забывай, что вы завтра в 12 танцуете',
 'В

In [122]:
mes = chatbot.clear_messages(all_messages)
mes

['пришли фотки пожалуйста .',
 'привет сейчас . всё',
 'спасибо большое . сережка перешли мне письмо смоленцевой . заранее спасибо .',
 'какое именно ?',
 'последнее . про кино . спасибо .',
 'скинул . подойдите завтра к смоленцевой .',
 'зачем ?',
 'она попросила .',
 'хорошо .',
 'юль . не забывай что вы завтра в танцуете . и своим передай .',
 'хорошо . спасибо . спасибо . а до скольких ? спасибо .']

In [123]:
chatbot.get_pairs(mes)

[['пришли фотки пожалуйста .', 'привет сейчас . всё'],
 ['какое именно ?', 'последнее . про кино . спасибо .'],
 ['последнее . про кино . спасибо .',
  'скинул . подойдите завтра к смоленцевой .'],
 ['скинул . подойдите завтра к смоленцевой .', 'зачем ?'],
 ['зачем ?', 'она попросила .'],
 ['она попросила .', 'хорошо .']]

In [24]:

def transform_message(message: str) -> str:
    """Make first char upper and add a dot in the end of sentence."""
    # TODO: normilise text

    if message:
        message = message[0].upper() + message[1:]
        if message[-1].isalpha():
            message += '.'
    return message

messages = []
last_author = all_messages[0][:all_messages[0].find(',')] # Человек, начавший беседу

for message in all_messages:
    author =  message[:message.find(',')]
    message = message[message.find("\n") + 1 :]
    clear_message = chatbot._clear_message(message)
    if not clear_message:
        last_author =  author 
        print('lol')
        continue
        
    if author == last_author: # still one message
        if messages:
            messages[-1] += (' ' + transform_message(clear_message))
        else: # for the first iteration
            messages.append(transform_message(clear_message))
    else: # if author change
        messages.append(transform_message(clear_message))

    last_author =  author   

        

lol
lol
lol


In [25]:
messages

['Пришли фотки, пожалуйста.',
 'Привет, сейчас. Всё👌',
 'Спасибо большое.',
 '👌',
 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
 'Какое именно?',
 'Последнее. Про кино. Спасибо.',
 'Скинул. Подойдите завтра к смоленцевой.',
 'Зачем?',
 'Она попросила.',
 'Хорошо.',
 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
 'Хорошо. Спасибо.',
 'Спасибо.',
 'А до скольких? Спасибо.']

In [33]:
messages

[':Юлия Николаева: пришли фотки, пожалуйста',
 ':Вы: Привет, сейчас',
 ':Вы: Всё👌',
 ':Юлия Николаева: Спасибо большое',
 ':Вы: 👌',
 ':Юлия Николаева: Сережка, перешли мне письмо Смоленцевой.',
 ':Юлия Николаева: Заранее спасибо',
 ':Вы: Какое именно?',
 ':Юлия Николаева: Последнее. Про кино',
 ':Юлия Николаева: Спасибо',
 ':Вы: Скинул',
 ':Вы: Подойдите завтра к смоленцевой',
 ':Юлия Николаева: зачем?',
 ':Вы: Она попросила',
 ':Юлия Николаева: Хорошо',
 ':Вы: Юль',
 ':Вы: не забывай, что вы завтра в 12 танцуете',
 ':Вы: И своим передай',
 ':Юлия Николаева: Хорошо',
 ':Юлия Николаева: спасибо',
 ':Юлия Николаева: Спасибо',
 ':Юлия Николаева: а до скольких?',
 ':Юлия Николаева: спасибо']

In [53]:
last_author = re.sub(': .*$', '', messages[0])[1:]
tmp = []

for i in messages:
    author = re.sub(': .*$', '', i)[1:]
    message = re.sub(':.*: ', '', i)
    
    if author == last_author: # значит это всё ещё сообщения от одного пользователя - tmp уже заполнен
        if tmp:
            tmp[-1] += (' ' + transform_message(message))
        else: # для первой итерации
            tmp.append(transform_message(message))
    else: # если пользователь сменился
        tmp.append(transform_message(message))
    
    last_author =  author   
        
tmp

['Пришли фотки, пожалуйста.',
 'Привет, сейчас. Всё👌',
 'Спасибо большое.',
 '👌',
 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
 'Какое именно?',
 'Последнее. Про кино. Спасибо.',
 'Скинул. Подойдите завтра к смоленцевой.',
 'Зачем?',
 'Она попросила.',
 'Хорошо.',
 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
 'Хорошо. Спасибо. Спасибо. А до скольких? Спасибо.']

In [63]:
[[tmp[i-1], tmp[i]] for i in range(1, len(tmp))]

[['Пришли фотки, пожалуйста.', 'Привет, сейчас. Всё👌'],
 ['Привет, сейчас. Всё👌', 'Спасибо большое.'],
 ['Спасибо большое.', '👌'],
 ['👌', 'Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.'],
 ['Сережка, перешли мне письмо Смоленцевой. Заранее спасибо.',
  'Какое именно?'],
 ['Какое именно?', 'Последнее. Про кино. Спасибо.'],
 ['Последнее. Про кино. Спасибо.', 'Скинул. Подойдите завтра к смоленцевой.'],
 ['Скинул. Подойдите завтра к смоленцевой.', 'Зачем?'],
 ['Зачем?', 'Она попросила.'],
 ['Она попросила.', 'Хорошо.'],
 ['Хорошо.', 'Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.'],
 ['Юль. Не забывай, что вы завтра в 12 танцуете. И своим передай.',
  'Хорошо. Спасибо. Спасибо. А до скольких? Спасибо.']]